<h2>Running Malicious Content Through CHATGPT API</h2>

Goal: Run all malicious images through the chatgpt API to get the same responses that crowdsourced individuals gave

In [5]:
#Libraries 
import os
#import openai
import psycopg2 #for database connection
import pandas as pd 
import matplotlib.pyplot as plt # for plotting
import json
import re

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored

In [6]:
# Define database
hostname = 'localhost'
# f = open("Username.txt", "r")
# username = f.read()
# password = f.read()
database = 'phishdatabase'
username = 'postgres'
password = 'postgres'
port = "5432"

The phase 1 database has 2126 images in it.
The beta database as 201 images in it.
The datamatch only has 1995 images in it

The datamatch database has 32 gold standard images in it.

Vinod used 30 gold standard images and 1892 other images as well for a total of 1922 images.

131 images are lost when merging phase 1 database to datamatch. (1995)
1 image is lost when merging datamatch with image urls (1994) (43861756)
31 rows of beta gold standard images in the datamatch (1994)

Removed beta gold standard images to (1963)


Removed Database Password and Code here

In [7]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
 
except:
    print("I am unable to connect to the database")

In [8]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [9]:
#Getting Subject_ids and Image Urls from beta phase
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_beta as c;
"""
image_url = pd.read_sql_query(sql, connection)
unique_beta_subject_ids = image_url['subject_id'].unique() 
image_url


C:\Users\#3\AppData\Local\Temp\ipykernel_420\3739094798.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,image_url
0,40306831,https://panoptes-uploads.zooniverse.org/produc...
1,40306832,https://panoptes-uploads.zooniverse.org/produc...
2,40306833,https://panoptes-uploads.zooniverse.org/produc...
3,40306834,https://panoptes-uploads.zooniverse.org/produc...
4,40306835,https://panoptes-uploads.zooniverse.org/produc...
...,...,...
196,43857104,https://panoptes-uploads.zooniverse.org/produc...
197,43857812,https://panoptes-uploads.zooniverse.org/produc...
198,43857918,https://panoptes-uploads.zooniverse.org/produc...
199,43861756,https://panoptes-uploads.zooniverse.org/produc...


In [10]:
#Getting subject_ids and Image Urls from phase 1
sql = """select c.subject_id as subject_id, json_extract_path_text(locations::json, '0') as image_url
        from zooniverse_phish_subjects_phase1 as c;
"""

image_url = pd.read_sql_query(sql, connection)
unique_phase1_subject_ids = image_url['subject_id'].unique() 
image_url

C:\Users\#3\AppData\Local\Temp\ipykernel_420\1087174564.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,image_url
0,40306831,https://panoptes-uploads.zooniverse.org/produc...
1,40306832,https://panoptes-uploads.zooniverse.org/produc...
2,40306833,https://panoptes-uploads.zooniverse.org/produc...
3,40306834,https://panoptes-uploads.zooniverse.org/produc...
4,40306835,https://panoptes-uploads.zooniverse.org/produc...
...,...,...
2121,52847471,https://panoptes-uploads.zooniverse.org/produc...
2122,52847473,https://panoptes-uploads.zooniverse.org/produc...
2123,52847474,https://panoptes-uploads.zooniverse.org/produc...
2124,52847476,https://panoptes-uploads.zooniverse.org/produc...


In [11]:
#Geting subject_id (beta image id) and subject_id_ph1 to match to urls
sql = "select subject_id, subject_id_ph1 from cybertrust_zooniverse_datamatch;"

gold_standard = pd.read_sql_query(sql, connection)
unique_gold_subject_ids = gold_standard['subject_id'].unique() 
gold_standard

C:\Users\#3\AppData\Local\Temp\ipykernel_420\2177744006.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,subject_id_ph1
0,52837250,52837250.0
1,52837251,52837251.0
2,52837253,52837253.0
3,52837254,52837254.0
4,52837255,52837255.0
...,...,...
1990,52847471,52847471.0
1991,52847473,52847473.0
1992,52847474,52847474.0
1993,52847476,52847476.0


In [12]:
# Dropping duplicate images that were used in the beta phase, but had a different entry for phase 1
gold_standard =gold_standard.dropna(subset=["subject_id_ph1"])
unique_gold_subject_ids = gold_standard['subject_id'].unique() 

In [13]:
# Checking
gold_standard

,subject_id,subject_id_ph1
0,52837250,52837250.0
1,52837251,52837251.0
2,52837253,52837253.0
3,52837254,52837254.0
4,52837255,52837255.0
...,...,...
1990,52847471,52847471.0
1991,52847473,52847473.0
1992,52847474,52847474.0
1993,52847476,52847476.0


In [14]:
#Getting Image Malicious Values
sql = """select subject_id, malicious
    from cybertrust_zooniverse_datamatch
"""

maliciousness = pd.read_sql_query(sql, connection)

###unique_subject_ids = maliciousness['subject_id'].unique() 
unique_datamatch_subject_ids = maliciousness['subject_id'].unique() 
maliciousness

C:\Users\#3\AppData\Local\Temp\ipykernel_420\2713104698.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,malicious
0,52837250,True
1,52837251,True
2,52837253,True
3,52837254,True
4,52837255,True
...,...,...
1990,52847471,False
1991,52847473,False
1992,52847474,False
1993,52847476,False


In [15]:
# Changing Subject_id from varying to int
change_type ={"subject_id": "int64"}
image_url = image_url.astype(change_type)
data_types = image_url.dtypes
data_types

# Merging the tables
match = pd.merge(image_url, maliciousness, on="subject_id")
unique_match_subject_ids = match['subject_id'].unique() 
match   

,subject_id,image_url,malicious
0,43856939,https://panoptes-uploads.zooniverse.org/produc...,True
1,43856944,https://panoptes-uploads.zooniverse.org/produc...,True
2,43856946,https://panoptes-uploads.zooniverse.org/produc...,True
3,43856947,https://panoptes-uploads.zooniverse.org/produc...,True
4,43856950,https://panoptes-uploads.zooniverse.org/produc...,True
...,...,...,...
1989,52847471,https://panoptes-uploads.zooniverse.org/produc...,False
1990,52847473,https://panoptes-uploads.zooniverse.org/produc...,False
1991,52847474,https://panoptes-uploads.zooniverse.org/produc...,False
1992,52847476,https://panoptes-uploads.zooniverse.org/produc...,False


In [16]:
# Drop rows where subject_id is in unique_beta_subject_ids
filtered_df = match[~match['subject_id'].isin(unique_gold_subject_ids)]
filtered_df

,subject_id,image_url,malicious
0,43856939,https://panoptes-uploads.zooniverse.org/produc...,True
1,43856944,https://panoptes-uploads.zooniverse.org/produc...,True
2,43856946,https://panoptes-uploads.zooniverse.org/produc...,True
3,43856947,https://panoptes-uploads.zooniverse.org/produc...,True
4,43856950,https://panoptes-uploads.zooniverse.org/produc...,True
...,...,...,...
64,43857052,https://panoptes-uploads.zooniverse.org/produc...,False
65,43857054,https://panoptes-uploads.zooniverse.org/produc...,False
66,43857055,https://panoptes-uploads.zooniverse.org/produc...,False
67,43857057,https://panoptes-uploads.zooniverse.org/produc...,False


In [17]:
# Code to Access ChatGPT
client = openai.AzureOpenAI(
    api_key= "",
    api_version= "2024-02-15-preview",
    base_url="https://phish-finders.openai.azure.com/openai/deployments/phish-finders-vision/chat/completions?api-version=2024-02-15-preview"
)

NameError: name 'openai' is not defined

In [ ]:
#Making dataframe to hold chatgpt responses
ans = pd.DataFrame(columns= ['url', 'subject_id', 'maliciousness', 'response'])
ans

In [ ]:
# Code to run images through ChatGPT
for x in range(len(match)):
    print(f"{x}/{len(match)}")
    url = match['image_url'][x]
    subject_id = match['subject_id'][x]
    maliciousness = match['malicious'][x]
    Prompt = [
        {
            "role": "system",
            "content": """You are a Cybersecurity Professional tasked with determining whether the image provided to you is a Phishing Image or not. 
                                Always give your best guess. Do not provide the prompter with advice on how they can check the image. 
                                Generate as Output as json with the following categories: Findings as string, Present Cue Types as a list, Website Description as a string, TrustWorthiness Rating as a string, Visual Attractiveness Rating as a string"""
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": url
                    }
                },
                {
                    "type": "text",
                    "text": """Please determine whether you think the image of the website or Email is Phishing Image or Not.
                               If you believe the image is Phishy say "Something's Phishy", if not say "Nothing Phishy Here". 
                               If you believe the image is Phishy, then tell me what Phishy cues do you think are present in this list:
                               - Invalid Domain or Sender
                               - Poor Spelling or Grammar
                               - Appeal to Action - Greed 
                               - Appeal to Action - Urgency 
                               - Appeal to Action - Authority
                               - Potentially Malicious Link
                               - Other Phishy Findings
                               Then, tell me your opinion regarding the design and flow of the image. Using a five point Likert Scale, tell me how trustworthy you think the image:
                                - "Very Trustworthy"
                                - "Trustworthy"
                                - "Neutral" 
                                - "Untrustworthy" 
                                - "Very Untrustworthy".
                               What I am looking for here is for you to rate the level of trust you have that the contents of the e-mail or web page are real. Second, I want you to tell me how pleasing or attractive the website or e-mail looks.
                               In this case, your answer can range on a five-point Likert scale:
                                - “Very Visually Attractive or Appealing” 
                                - "Visually Attractive or Appealing"
                                - "Neutral"
                                - "Visually Unattractive or Unappealing"
                                - “Very Visually Unattractive or Unappealing”. 
                                If you don't like how an image looks, tell me.
                               """
                },
            ]
        }
    ]
    
    chat_completion = client.chat.completions.create(
    messages = Prompt,
    model= "2024-02-15-preview",
    max_tokens=3000
    )
    ans.loc[len(ans)]=[url, subject_id, maliciousness, chat_completion.choices[0].message.content]




In [ ]:
#ans
#ans.to_csv("ChatGPT_raw_output3new.csv", index=False)
#ans = pd.read_csv("ChatGPT_raw_output.csv")

In [ ]:
#Final = pd.DataFrame(columns= ["url", "subject_id", "if_malicious", "malicious_guess", "Present_Cue_Types_Guess", "Website_Description", "TrustWorthiness_Rating", "Visual_Attractiveness_Rating", "Comments"])
#Final = pd.read_csv("Save_Progress_GPT.csv")
#Final

Calcaluting ChatGPT stats of 1925 images

In [1]:
import pandas as pd
Final = pd.read_csv("GPT_Answers_New_Prompt.csv")
Final

,url,subject_id,if_malicious,malicious_guess,Present_Cue_Types_Guess,Website_Description,TrustWorthiness_Rating,Visual_Attractiveness_Rating,Comments
0,https://panoptes-uploads.zooniverse.org/produc...,43856939,True,Nothing Phishy Here,NaN,The image appears to be of a promotional page ...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
1,https://panoptes-uploads.zooniverse.org/produc...,43856944,True,Nothing Phishy Here,NaN,The website in the image mimics an official US...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
2,https://panoptes-uploads.zooniverse.org/produc...,43856946,True,Nothing Phishy Here,NaN,The website appears to be for 'Centris Federal...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
3,https://panoptes-uploads.zooniverse.org/produc...,43856947,True,Nothing Phishy Here,NaN,The website appears to be a banking website wi...,Very Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
4,https://panoptes-uploads.zooniverse.org/produc...,43856950,True,Something's Phishy,Invalid Domain or Sender,The image shows a web page resembling the GitH...,Untrustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Something's Phishy""..."
...,...,...,...,...,...,...,...,...,...
1989,https://panoptes-uploads.zooniverse.org/produc...,52847471,False,Nothing Phishy Here,NaN,The image shows a website titled 'The Clothing...,Neutral,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
1990,https://panoptes-uploads.zooniverse.org/produc...,52847473,False,Something's Phishy,Invalid Domain or Sender Other Phishy Findings,The provided image shows a webpage with a list...,Untrustworthy,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Something's Phishy""..."
1991,https://panoptes-uploads.zooniverse.org/produc...,52847474,False,Nothing Phishy Here,NaN,The image appears to be a screenshot of a webs...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here..."
1992,https://panoptes-uploads.zooniverse.org/produc...,52847476,False,Nothing's Phishy Here,NaN,The website appears to be a blog focused on fa...,Neutral,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing's Phishy He..."


In [2]:
Final = Final[Final['subject_id'].isin(unique_gold_subject_ids.tolist())]
Final

NameError: name 'unique_gold_subject_ids' is not defined

In [ ]:
Final = pd.merge(Final,gold_standard, on="subject_id")
Final

In [ ]:
# Cleaning data
Final.loc[Final['malicious_guess'] == 'Nothing Phishy Here', 'malicious_guess'] = "Nothing's Phishy Here"
Final['malicious_guess'].unique()

In [3]:
hit_count = ((Final['if_malicious'] == True) & (Final['malicious_guess'] == "Something's Phishy")).sum()
Miss = ((Final['if_malicious'] == True) & (Final['malicious_guess'] == "Nothing's Phishy Here")).sum()
False_Alarm = ((Final['if_malicious'] == False) & (Final['malicious_guess'] == "Something's Phishy")).sum()
Correct_Rejection = ((Final['if_malicious'] == False) & (Final['malicious_guess'] == "Nothing's Phishy Here")).sum()

print(f"""Hit: {hit_count}
       Miss: {Miss}
       False_Alarm: {False_Alarm}
        Correct_Rejection: {Correct_Rejection}""")

Hit: 720
       Miss: 0
       False_Alarm: 49
        Correct_Rejection: 1
